In [199]:
import pandas as pd
from scipy import stats
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from sentence_transformers import SentenceTransformer
import torch
from torch import nn
from collections import defaultdict
from pprint import pprint
import pickle

RANDOM_SEED=42
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

## Translation Model -> SBERT -> Pretrained

In [200]:
# Code
# SOURCE: Telugu	tel_Telu
# TARGET: English	eng_Latn

In [306]:
TRANSLATION_MODELS = [
    # ("google/madlad400-3b-mt", 16),
    # ("google/madlad400-10b-mt", 16),
    # ("facebook/mbart-large-50-many-to-many-mmt", 16),
    # ("facebook/mbart-large-50-many-to-one-mmt", 16),
    # ("facebook/mbart-large-50-one-to-many-mmt", 16),
    # ("facebook/mbart-large-50", 16),
    # ("facebook/mbart-large-cc25", 16),
    ("facebook/nllb-200-3.3B", 8),
    ("facebook/nllb-200-1.3B", 16),
    ("facebook/nllb-200-distilled-600M", 16), # bsz fixed
    ("facebook/nllb-200-distilled-1.3B", 16), # bsz fixed
    # ("facebook/nllb-moe-54b", 2),
]

In [202]:
STS_MODELS = [
    "sentence-transformers/sentence-t5-xxl",
    "sentence-transformers/gtr-t5-xxl",
    "sentence-transformers/all-roberta-large-v1",
    "sentence-transformers/all-mpnet-base-v1",
    "sentence-transformers/gtr-t5-large",
    "sentence-transformers/gtr-t5-xl",
    "sentence-transformers/all-mpnet-base-v2",
    "sentence-transformers/sentence-t5-xl",
    "sentence-transformers/sentence-t5-large",
    "sentence-transformers/all-MiniLM-L12-v1",
    "sentence-transformers/all-distilroberta-v1",
    "sentence-transformers/all-MiniLM-L12-v2",
    "sentence-transformers/all-MiniLM-L6-v2",   
]

UNSUP_MODELS = [
    "bert-base-uncased",
    "bert-large-uncased",
    "roberta-base",
    "roberta-large",
]

In [203]:
!ls data/"Track A"/

amh  arq  ary  eng  esp  hau  kin  mar	tel


In [148]:
!wc -l "data/Track A/tel/tel_train.csv"

2341 data/Track A/tel/tel_train.csv


In [143]:
df = pd.read_csv("./data/Track A/tel/tel_train.csv")
df["text1"] = df["Text"].map(lambda x: x.split("\n")[0].strip('"'))
df["text2"] = df["Text"].map(lambda x: x.split("\n")[1].strip('"'))

In [144]:
df.head()

,PairID,Text,Score,text1,text2
0,kin_train_00001,Izi serivisi benshi bigaragara ko batarasobanu...,0.31,Izi serivisi benshi bigaragara ko batarasobanu...,"Col Dr Charles Furaha, ni umusirikare w’umugan..."
1,kin_train_00002,Avuga ko agahinda ke katangiye gukura ubwo yar...,0.31,Avuga ko agahinda ke katangiye gukura ubwo yar...,"Iyo akiri umwana aba avuga utugambo twinshi, u..."
2,kin_train_00003,Igitego cya Rayon Sports cyatsinzwe n'umusore ...,0.44,Igitego cya Rayon Sports cyatsinzwe n'umusore ...,Rayon Sports itsinda igitego cya mbere ku muno...
3,kin_train_00004,Ubutumwa buri muri iyi ndirimbo ni ukumenya no...,0.57,Ubutumwa buri muri iyi ndirimbo ni ukumenya no...,Ndayishimiye yavuze ko ubutumwa buri muri iyi ...
4,kin_train_00005,Iwacu w’imyaka 20 y’amavuko yiga iby’ikoranabu...,0.50,Iwacu w’imyaka 20 y’amavuko yiga iby’ikoranabu...,Ubu yiga mu mwaka wa nyuma mu ishami ry’ikoran...


In [145]:
cos = nn.CosineSimilarity(dim=-1, eps=1e-6)

In [16]:
all_translations = defaultdict(dict)

In [146]:
@torch.no_grad()
def get_transolations(tmodel_name, df, batch_size=16, split=1.0):
    tmodel = AutoModelForSeq2SeqLM.from_pretrained(tmodel_name)
    ttokenizer = AutoTokenizer.from_pretrained(tmodel_name)
    source = "tel_Telu"
    target = "eng_Latn"
    task_name = 'translation'
    # if tmodel_name.index("mbart") != -1: task_name = "translation_te_to_en"
    translator = pipeline(task_name, model=tmodel, tokenizer=ttokenizer, src_lang=source, tgt_lang=target, batch_size=batch_size, device=DEVICE)

    df = df.sample(frac=split, random_state=RANDOM_SEED)

    texts = []
    for i, row in df.iterrows():
        text1 = row['text1']
        text2 = row['text2']
        texts.append(text1)
        texts.append(text2)
    translations = translator(texts, max_length=400)
    translations = [x['translation_text'] for x in translations]

    nw_translation = {k: v for k, v in zip(texts, translations)}

    print(texts[0])
    print(translations[0])

    return df, translations

@torch.no_grad()
def get_output(translations, smodel_name, df, test=False):
    smodel = SentenceTransformer(smodel_name)
    smodel = smodel.to(DEVICE)

    embs = smodel.encode(translations, convert_to_tensor=True, device=DEVICE)
    emb1 = []
    emb2 = []
    for i in range(0, embs.size(0), 2):
        emb1.append(embs[i, :])
        emb2.append(embs[i+1, :])
    emb1 = torch.stack(emb1).float()
    emb2 = torch.stack(emb2).float()
    
    y_hat = cos(emb1, emb2)

    if test: return y_hat
    y = torch.tensor(df['Score'].tolist())
    score = stats.spearmanr(y.cpu().numpy(), y_hat.cpu().numpy())[0]
    return score

In [147]:
results = defaultdict(dict)

for tmodel, bsz in TRANSLATION_MODELS:
    try:
        print(f"******* {tmodel} *******")
        curr_df, translations = get_transolations(tmodel, df, batch_size=bsz, split=1.0)  
        for smodel in STS_MODELS:
            curr_res = get_output(translations, smodel, curr_df)
            result = f"{tmodel}".ljust(70) + " " + f"{smodel}".ljust(70) + " " + str(curr_res)
            print(result)
            results[tmodel][smodel] = curr_res
    except Exception as e:
        print("**** ERROR ****")
        print(tmodel, smodel)
        print(e)
    finally:
        print()

******* facebook/nllb-200-distilled-600M *******
Umunyamabanga nshingwabikorwa wa CNLG, Dr Bizimana Jean Damascène nawe yari ahari.
The CNLG's deputy secretary, Dr Bizimana Jean Damascène, was also present.
facebook/nllb-200-distilled-600M                                       sentence-transformers/sentence-t5-xxl                                  0.48022211816185967



KeyboardInterrupt: 

In [21]:
pprint(results)

defaultdict(<class 'dict'>,
            {'facebook/mbart-large-50-many-to-many-mmt': {'sentence-transformers/all-MiniLM-L12-v1': 0.48226540531804185,
                                                          'sentence-transformers/all-MiniLM-L12-v2': 0.49439394393661734,
                                                          'sentence-transformers/all-MiniLM-L6-v2': 0.4869803048674303,
                                                          'sentence-transformers/all-distilroberta-v1': 0.5040714737715142,
                                                          'sentence-transformers/all-mpnet-base-v1': 0.4661510433130261,
                                                          'sentence-transformers/all-mpnet-base-v2': 0.46543653653745115,
                                                          'sentence-transformers/all-roberta-large-v1': 0.5110120223357795,
                                                          'sentence-transformers/gtr-t5-large': 0.519788632834131,
  

In [42]:
with open("./translations.pkl", 'rb') as fp:
    all_translations = pickle.load(fp)

len(all_translations)

5

In [43]:
!ls ./data/"Track A"/

amh  arq  ary  eng  esp  hau  kin  mar	tel


In [82]:
dev_df = pd.read_csv("./data/Track A/tel/tel_dev.csv")
dev_df["text1"] = dev_df["Text"].map(lambda x: x.split("\n")[0].strip('"'))
dev_df["text2"] = dev_df["Text"].map(lambda x: x.split("\n")[1].strip('"'))

In [83]:
dev_df.head()

,PairID,Text,text1,text2
0,TEL-dev-00001,"""బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరో...",బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా నేడు రెండోరోజ...,బీజేపీ జాతీయ అధ్యక్షుడు అమిత్ షా గారు ఈ రోజు ర...
1,TEL-dev-00002,"""ఈ ఏడాది జనవరి 1వ తేదీ నాటికి ఢిల్లీలో 10,76,4...","ఈ ఏడాది జనవరి 1వ తేదీ నాటికి ఢిల్లీలో 10,76,46...",ఈ ఏడాది చివరి నాటికల్లా కంపెనీ ఈ అప్పుల్లో కొం...
2,TEL-dev-00003,"""ప్రతిపక్షాలు అనవసర రాద్ధాంతం చేస్తున్నాయి : హ...",ప్రతిపక్షాలు అనవసర రాద్ధాంతం చేస్తున్నాయి : హర...,హైదరాబాద్ : తెలంగాణ అభివృద్ధిని అడ్డుకునేందుకు...
3,TEL-dev-00004,"""హైదరాబాద్ : కాంగ్రెస్ పార్టీ అధికారంలోకి రాగా...",హైదరాబాద్ : కాంగ్రెస్ పార్టీ అధికారంలోకి రాగాన...,ఇందిరమ్మ ఇళ్లలో అక్రమాలు జరిగాయని టిఆర్ఎస్ నాయ...
4,TEL-dev-00005,"""ప్రజారాజధానిగా రూపుదిద్దేందుకు అవసరమైన పెట్టు...",ప్రజారాజధానిగా రూపుదిద్దేందుకు అవసరమైన పెట్టుబ...,ఈ ఒప్పం దం తక్షణమే అమల్లోకి వస్తుందని సంబంధిత ...


In [131]:
results = defaultdict(dict)

for tmodel, bsz in TRANSLATION_MODELS:
    try:
        print(f"******* {tmodel} *******")
        curr_df, translations = get_transolations(tmodel, dev_df, batch_size=bsz, split=1.0)
        for smodel in STS_MODELS[1:]:
            out = get_output(translations, smodel, curr_df, test=True)
            print(out.shape)
            break
    except Exception as e:
        print("**** ERROR ****")
        print(tmodel, smodel)
        print(e)
    finally:
        print()
        break

******* facebook/nllb-200-distilled-600M *******
బాలీవుడ్ నటి ఆలియా భట్ తన పార్టీ గుర్తును ప్రకటించారు . ఎందుకంటే ఎక్కడికెళ్లినా ఈ మధ్య రాజకీయాల ప్రస్తావన ఆమెను వదిలి పెట్టడం లేదు .
Bollywood actress Alia Bhatt has announced her party logo because nowhere she goes is a political reference to her.
torch.Size([130])



In [134]:
curr_df['Pred_Score'] = out.tolist()
curr_df

,PairID,Text,text1,text2,Pred_Score
55,TEL-dev-00056,"""బాలీవుడ్ నటి ఆలియా భట్ తన పార్టీ గుర్తును ప్ర...",బాలీవుడ్ నటి ఆలియా భట్ తన పార్టీ గుర్తును ప్రక...,హిందీ నటి అలియా భట్ పార్టీ గుర్తును ప్రకటించిం...,0.942245
40,TEL-dev-00041,"""నీరవ్ మోదీ ఆయన బంధువు మెహుల్ చోక్సీలకు చెందిన...",నీరవ్ మోదీ ఆయన బంధువు మెహుల్ చోక్సీలకు చెందిన ...,స్వేచ్చాయుత వాణిజ్య ఒప్పందాల వల్ల ఇరు దేశాల మధ...,0.491731
19,TEL-dev-00020,"""ఈ సందర్భంగా కోహ్లి, అనుష్క బంధువులు, స్నేహితు...","ఈ సందర్భంగా కోహ్లి, అనుష్క బంధువులు, స్నేహితుల...",ఈ వీడియోను చెన్నై సూపర్ కింగ్స్ ట్విట్టర్ అకౌం...,0.603082
31,TEL-dev-00032,"""ఢిల్లీ దబాంగ్తో జరిగిన మ్యాచ్లో పాట్నా 13-33త...",ఢిల్లీ దబాంగ్తో జరిగిన మ్యాచ్లో పాట్నా 13-33తో...,డిఫెన్స్లో ఓ పట్టు పట్టిన ఢిల్లీ వరుస విజయాలతో...,0.757032
115,TEL-dev-00116,"""ఎపి రాజధాని అమరావతి శంకుస్థాపనకు రావాలని బాబు...",ఎపి రాజధాని అమరావతి శంకుస్థాపనకు రావాలని బాబు ...,25 చొప్పున సరఫరా చేస్తున్నామని. . అలాగే కేంద్ర...,0.523448
...,...,...,...,...,...
71,TEL-dev-00072,"""అనుభవం కలిగిన ఆటగాడిగా ధోనీ తనకు సూచనలు చేసే ...",అనుభవం కలిగిన ఆటగాడిగా ధోనీ తనకు సూచనలు చేసే వ...,"ఇక, వికెట్ల వెనుక ఉంటూ ధోనీ బౌలర్లతో వ్యూహాలు ...",0.726495
106,TEL-dev-00107,"""కృష్ణ జింకల వేట కేసు : సల్మాన్ దోషి""\n""జోథ్పూ...",కృష్ణ జింకల వేట కేసు : సల్మాన్ దోషి,జోథ్పూర్ : కృష్ణ జింకలను వేటాడిన కేసులో బాలీవు...,0.659634
14,TEL-dev-00015,"""ఈ ఒప్పందాలవల్ల సుమారుగా 2.01 లక్షలమందికి ఉపాధ...",ఈ ఒప్పందాలవల్ల సుమారుగా 2.01 లక్షలమందికి ఉపాధి...,అలాగే ప్రస్తుతం ఢిల్లీ-విజయవాడ మధ్య వారానికి మ...,0.477149
92,TEL-dev-00093,"""రెండో టెస్టులో ఆసీస్పై ప్రతికారం తీర్చుకునేంద...",రెండో టెస్టులో ఆసీస్పై ప్రతికారం తీర్చుకునేందు...,మార్చి 4న బెంగళూరులోని చిన్నస్వామి స్టేడియంలో ...,0.711676


In [135]:
import os
os.remove('./out/pred_tel_a.csv')
curr_df.to_csv('./out/pred_tel_a.csv', index=False, columns=['PairID', 'Pred_Score'])

In [136]:
!ls out

pred_tel_a.csv


In [137]:
tmp = pd.read_csv('./out/pred_tel_a.csv')

In [138]:
list(tmp.columns), len(tmp.columns)

(['PairID', 'Pred_Score'], 2)

In [139]:
tmp.head()

,PairID,Pred_Score
0,TEL-dev-00056,0.942245
1,TEL-dev-00041,0.491731
2,TEL-dev-00020,0.603082
3,TEL-dev-00032,0.757032
4,TEL-dev-00116,0.523448


# Subtask B

In [154]:
!ls data/"Track A"/

amh  arq  ary  eng  esp  hau  kin  mar	tel


In [130]:
!head -10 data/"Track C"/hin/hin_dev.csv

PairID,Text
HIN-pilot-00001,"""जॉर्जियाई अधिकारियों ने बताया कि 8 अगस्त 2008 को पोटी के काला सागर बंदरगाह पर लगभग 1 हवाई हमला तो हुआ था।""
""2008 में जॉर्जिया के लोगों ने बताया कि 8 अगस्त को पोटी नाम की जगह पर हवाई जहाज़ से हमला हुआ था. पोटी काला सागर के पास एक बंदरगाह है।"""
HIN-pilot-00002,"""पुलिस आरोपित से अवैध हथियार की खरीद-फरोख्त के बारे में पूछताछ कर ही है।""
""पर हमें दीर्घावधि के दृष्टिकोण से सोचना होगा।"""
HIN-pilot-00003,"""कई बार पहले भी बच्ची को जान से मारने की धमकी दे चुका था।""
""इसी बात के विवाद में वह मायके भी चली गई थी।"""
HIN-pilot-00004,"""धुंध से परेशान हैं, हिमाचल की पहाडियों का रूख करें""
""बोस ने कहा कि हो सकता है कि कोई ऐसा कर रहा हो।"""
HIN-pilot-00005,"""इस दौरान वह दो बार गर्भवती हुई और चार बार उसने खुदकुशी की कोशिश भी की।""


In [156]:
dev_df = pd.read_csv("data/Track B/hin/hin_dev.csv")
dev_df["text1"] = dev_df["Text"].map(lambda x: x.split("\n")[0].strip('"'))
dev_df["text2"] = dev_df["Text"].map(lambda x: x.split("\n")[1].strip('"'))
dev_df.head()

,PairID,Text,text1,text2
0,HIN-dev-00001,"""जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को ...",जालपुरा में बुधवार को फर्जी एसीबी अधिकारी को अ...,आरोपी एसीबी अधिकारी बनकर लोगो से चौथ वसूली कर ...
1,HIN-dev-00002,"""राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने ...","राज्यपाल नहीं पहुंच सके, इसलिए उपराज्यपाल ने व...","राज्यपाल उपलब्ध नहीं थे, इसलिए उपराज्यपाल ने म..."
2,HIN-dev-00003,"""बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वरु...","बात कर रहे हैं फिल्म 'दिलवाले' की, जिसमें वरुण...",अभिनेता वरुण धवन ने कहा है कि अभिनेता-पहलवान ड...
3,HIN-dev-00004,"""घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र की...",घटना बेगूसराय के नीमाचांदपुरा थाना क्षेत्र की है।,प्राप्त जानकारी के अनुसार घटना डरबन में घटित ह...
4,HIN-dev-00005,"""शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियों...",शिकायत मिलने पर छावनी पुलिस ने तीनों आरोपियों ...,"पुलिस ने मामला दर्ज कर लिया है, लेकिन अभी तक क..."


In [158]:
results = defaultdict(dict)

for tmodel, bsz in TRANSLATION_MODELS:
    try:
        print(f"******* {tmodel} *******")
        curr_df, translations = get_transolations(tmodel, dev_df, batch_size=bsz, split=1.0)
        for smodel in UNSUP_MODELS:
            out = get_output(translations, smodel, curr_df, test=True)
            print(out.shape)
            break
    except Exception as e:
        print("**** ERROR ****")
        print(tmodel, smodel)
        print(e)
    finally:
        print()
        break

******* facebook/nllb-200-distilled-600M *******
कोतवाली इलाके में शादी का झांसा देकर एक महिला से देहशोषण करने का मामला सामने आया है।
A woman has been charged with sexual assault in the Kotwali area.


No sentence-transformers model found with name /home/sroydip1/.cache/torch/sentence_transformers/bert-base-uncased. Creating a new one with MEAN pooling.


torch.Size([288])



In [160]:
import os
curr_df['Pred_Score'] = out.tolist()
# os.remove('./out/pred_hin_b.csv')
curr_df.to_csv('./out/pred_hin_b.csv', index=False, columns=['PairID', 'Pred_Score'])

In [161]:
tmp = pd.read_csv('./out/pred_hin_b.csv')
tmp.head()

,PairID,Pred_Score
0,HIN-dev-00046,0.739316
1,HIN-dev-00158,0.663134
2,HIN-dev-00257,0.696460
3,HIN-dev-00043,0.644122
4,HIN-dev-00182,0.642022


In [172]:
os.listdir("./data/Track A/")

['amh', 'arq', 'ary', 'eng', 'esp', 'hau', 'kin', 'mar', 'tel']

In [190]:
lang_codes = {
    "amh": "amh_Ethi",
    "ary": "ary_Arab",
    "eng": "eng_Latn",
    "esp": "spa_Latn",
    "hau": "hau_Latn",
    "kin": "kin_Latn",
    "mar": "mar_Deva",
    "tel": "tel_Telu"
}

TRANSLATION_MODELS = [
    # ("google/madlad400-3b-mt", 16),
    # ("google/madlad400-10b-mt", 16),
    # ("facebook/mbart-large-50-many-to-many-mmt", 16),
    # ("facebook/mbart-large-50-many-to-one-mmt", 16),
    # ("facebook/mbart-large-50-one-to-many-mmt", 16),
    # ("facebook/mbart-large-50", 16),
    # ("facebook/mbart-large-cc25", 16),
    ("facebook/nllb-200-1.3B", 16),
    ("facebook/nllb-200-3.3B", 8),
    ("facebook/nllb-200-distilled-600M", 16), # bsz fixed
    ("facebook/nllb-200-distilled-1.3B", 16), # bsz fixed
    # ("facebook/nllb-moe-54b", 2),
]

In [238]:
os.listdir("./data/Track A/")

['amh', 'arq', 'ary', 'eng', 'esp', 'hau', 'kin', 'mar', 'tel']

In [311]:
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from collections import defaultdict
from pathlib import Path
Path("./data/translations/").mkdir(exist_ok=True)

tot_train = 0
tot_val = 0

for lang in os.listdir("./data/Track A/"):
    if lang in ["amh", "arq"]: continue
    if not os.path.isdir(f"./data/Track A/{lang}"): continue

    df = pd.read_csv(f"./data/Track A/{lang}/{lang}_train.csv")
    train_df, val_df = train_test_split(df, test_size=0.1, random_state=RANDOM_SEED)
    train_df.to_csv(f"./data/Track A/{lang}/{lang}_train_split.csv")
    val_df.to_csv(f"./data/Track A/{lang}/{lang}_val_split.csv")

    def write_translation(mode):
        file_name = f"{lang}_{mode}"
        if mode != "dev": file_name += '_split'
        df = pd.read_csv(f"./data/Track A/{lang}/{file_name}.csv")
        df["text1"] = df["Text"].map(lambda x: x.split("\n")[0].strip('"'))
        df["text2"] = df["Text"].map(lambda x: x.split("\n")[1].strip('"'))
    
        print(lang, mode, len(df))
    
        all_translations = defaultdict(list)
        for tmodel_name, batch_size in tqdm(TRANSLATION_MODELS):
            tmodel = AutoModelForSeq2SeqLM.from_pretrained(tmodel_name)
            ttokenizer = AutoTokenizer.from_pretrained(tmodel_name)
            source = lang_codes[lang]
            target = "eng_Latn"
            task_name = 'translation'
            # if tmodel_name.index("mbart") != -1: task_name = "translation_te_to_en"
            translator = pipeline(task_name, model=tmodel, tokenizer=ttokenizer, src_lang=source, tgt_lang=target, batch_size=batch_size, device=DEVICE)
        
            texts1 = []
            texts2 = []
            for i, row in df.iterrows():
                text1 = row['text1']
                text2 = row['text2']
                texts1.append(text1)
                texts2.append(text2)
            translations1 = translator(texts1, max_length=800)
            translations1 = [x['translation_text'] for x in translations1]
    
            translations2 = translator(texts2, max_length=800)
            translations2 = [x['translation_text'] for x in translations2]

            for i, (_, row) in enumerate(df.iterrows()):
                all_translations['text1'].append(translations1[i])
                all_translations['text2'].append(translations2[i])
                all_translations['PairID'].append(row['PairID'])
                all_translations['model'].append(tmodel_name)
                if mode != 'dev':
                    all_translations['Score'].append(row['Score'])

            if lang == 'eng': break
    
        out_df = pd.DataFrame(all_translations)
        out_df.to_csv(f"./data/Track A/{lang}/{mode}_translation.csv")

    for mode in ['train', 'val', 'dev']:
        write_translation(mode)

eng
eng train 4950


  0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

eng val 550


  0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

eng dev 250


  0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [294]:
tot_train, tot_val

(0, 0)

In [312]:
!ls

ada.sh		   jupyter.sh  pyproject.toml  sweep.sh		 wandb
data		   LICENSE     README.md       sweep.yaml
evaluation_script  log	       run	       train.sh
exploration.ipynb  out	       src	       translations.pkl


In [313]:
dirs = [
    # 'amh',
    # 'arq',
    'ary', 'eng', 'esp', 'hau', 'kin', 'mar', 'tel']

In [314]:
all_trains = []
all_vals = []
all_devs = []

for d in dirs:
    df = pd.read_csv(f"./data/Track A/{d}/train_translation.csv")
    df['lang'] = d
    all_trains.append(df)

    df = pd.read_csv(f"./data/Track A/{d}/val_translation.csv")
    df['lang'] = d
    all_vals.append(df)

    df = pd.read_csv(f"./data/Track A/{d}/dev_translation.csv")
    df['lang'] = d
    all_devs.append(df)

train = pd.concat(all_trains)
val = pd.concat(all_vals)
dev = pd.concat(all_devs)

train.to_csv("./data/Track A/train_all.csv")
val.to_csv("./data/Track A/val_all.csv")
dev.to_csv("./data/Track A/dev_all.csv")

In [315]:
len(train)

31474

In [316]:
train.head()

,Unnamed: 0,text1,text2,PairID,model,Score,lang
0,0,The temperature will be 47 degrees from tomorr...,They found a rascomb in the rainbow. The tempe...,ARY-train-0881,facebook/nllb-200-1.3B,0.69,ary
1,1,Corona outbreak today: 7805 people have receiv...,"Corona and vaccines: 132 cases, 7 deaths, and ...",ARY-train-0096,facebook/nllb-200-1.3B,0.50,ary
2,2,Belgium at Security Council: UN resolves Sahar...,Desert workers for peace sends papal messages ...,ARY-train-0371,facebook/nllb-200-1.3B,0.44,ary
3,3,Gendarmes have shut down an international smug...,A major blow to security forces in the north: ...,ARY-train-0066,facebook/nllb-200-1.3B,0.57,ary
4,4,Eyes.. come and in coordination with the Dusty...,"In coordination with the police, more than hal...",ARY-train-0342,facebook/nllb-200-1.3B,0.53,ary


In [317]:
len(val)

3506

In [318]:
val.head()

,Unnamed: 0,text1,text2,PairID,model,Score,lang
0,0,The Directorate General of National Security p...,The case of the attack on the German tourist.....,ARY-train-0323,facebook/nllb-200-1.3B,0.43,ary
1,1,Today's reality: the result of today's crimes ...,Today's facts: The result of this day of crime...,ARY-train-0861,facebook/nllb-200-1.3B,0.47,ary
2,2,Protests over traffic violation. Young man set...,Fire is still a means of protest: A young man ...,ARY-train-0030,facebook/nllb-200-1.3B,0.39,ary
3,3,After he died.. the judiciary dismissed the pu...,Extending the theoretical custody of parliamen...,ARY-train-0837,facebook/nllb-200-1.3B,0.17,ary
4,4,Finance . People: 91 billion dirhams allocated...,"More than 21,000 jobs at the shield of Tavilal...",ARY-train-0294,facebook/nllb-200-1.3B,0.62,ary


In [319]:
len(dev)

4042

In [320]:
dev.head()

,Unnamed: 0,text1,text2,PairID,model,lang
0,0,This is the first photo of Tamer Hosni's daugh...,The first dispute between Tamer Hosni and his ...,ARY-dev-0000,facebook/nllb-200-1.3B,ary
1,1,The case of hemsha Mon Pepe is a big one. The ...,"Her horse brought her a ""Hamza Mon Bebe"" that ...",ARY-dev-0001,facebook/nllb-200-1.3B,ary
2,2,They stayed in Diorcom. 38 new coronavirus cas...,Stay in your ward . We are doing well . 37 new...,ARY-dev-0002,facebook/nllb-200-1.3B,ary
3,3,The BJP wants to hold a mass vote in the elect...,The BJD wants to run a clean protest against t...,ARY-dev-0003,facebook/nllb-200-1.3B,ary
4,4,Corona today: 2 dead and 151 infected and 1 mi...,"Corona: 8338 infected and 10 dead in 24 hours,...",ARY-dev-0004,facebook/nllb-200-1.3B,ary


In [321]:
dev['model'].value_counts()

model
facebook/nllb-200-3.3B              1198
facebook/nllb-200-1.3B               948
facebook/nllb-200-distilled-600M     948
facebook/nllb-200-distilled-1.3B     948
Name: count, dtype: int64